In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Live MA Crossover Alerts (Batch-Optimized for 50 NSE tickers)
-------------------------------------------------------------
- Single batch request for intraday (1m) data across all symbols per poll.
- Single batch request for daily history across all symbols per refresh.
- Detects intraday price crossing DAILY MAs (20/50/100/150/200 by default).
- Telegram alerts on true cross up/down with debouncing.

Requirements: pip install yfinance requests pytz (if Python<3.9)
Run: python live_ma_batch_alerts.py   (during 09:15–15:30 IST)

Notes:
- Yahoo limits are soft; batching drastically reduces calls.
- If throttled (HTTP 429 / empty frames), script backs off and resumes.
"""

import os, json, time, math, logging, random
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Tuple
from datetime import datetime, timedelta

import pandas as pd
import yfinance as yf
import requests

try:
    from zoneinfo import ZoneInfo
    IST = ZoneInfo("Asia/Kolkata")
except Exception:
    import pytz
    IST = pytz.timezone("Asia/Kolkata")

# =========================
# CONFIG
# =========================
@dataclass
class Config:
    SYMBOLS: List[str] = field(default_factory=lambda: [
        'BSE.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BDL.NS', 'BEL.NS', 'BHARTIARTL.NS', 'CHOLAFIN.NS', 'COFORGE.NS', 'DIVISLAB.NS', 'DIXON.NS', 'NYKAA.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'ICICIBANK.NS', 'INDHOTEL.NS', 'INDIGO.NS', 'KOTAKBANK.NS', 'MFSL.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MUTHOOTFIN.NS', 'PAYTM.NS', 'PERSISTENT.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SRF.NS', 'SHREECEM.NS', 'SOLARINDS.NS', 'TVSMOTOR.NS', 'UNITDSPR.NS', 'ADANIENT.NS', 'ADANIPORTS.NS', 'APOLLOHOSP.NS', 'ASIANPAINT.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BEL.NS', 'BHARTIARTL.NS', 'CIPLA.NS', 'COALINDIA.NS', 'DRREDDY.NS', 'EICHERMOT.NS', 'ETERNAL.NS', 'GRASIM.NS', 'HCLTECH.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HINDALCO.NS', 'HINDUNILVR.NS', 'ICICIBANK.NS', 'ITC.NS', 'INFY.NS', 'INDIGO.NS', 'JSWSTEEL.NS', 'JIOFIN.NS', 'KOTAKBANK.NS', 'LT.NS', 'M&M.NS', 'MARUTI.NS', 'MAXHEALTH.NS', 'NTPC.NS', 'NESTLEIND.NS', 'ONGC.NS', 'POWERGRID.NS', 'RELIANCE.NS', 'SBILIFE.NS', 'SHRIRAMFIN.NS', 'SBIN.NS', 'SUNPHARMA.NS', 'TCS.NS', 'TATACONSUM.NS', 'TATAMOTORS.NS', 'TATASTEEL.NS', 'TECHM.NS', 'TITAN.NS', 'TRENT.NS', 'ULTRACEMCO.NS', 'WIPRO.NS'
    ])
    MAS: List[int] = field(default_factory=lambda: [150, 200])
    POLL_SECONDS: int = 120           # intraday poll interval (batch 1m)
    EPS_BAND: float = 0.001           # 0.1% neutral band to reduce noise
    STATE_FILE: str = "ma_cross_state.json"
    LOG_LEVEL: int = logging.INFO

    # Daily history refresh cadence
    DAILY_LOOKBACK_DAYS: int = 450    # enough for 200DMA
    DAILY_REFRESH_MIN: int = 30

    # Telegram
    TELEGRAM_ENABLED: bool = False
    TELEGRAM_BOT_TOKEN: str = "PUT_YOUR_BOT_TOKEN"
    TELEGRAM_CHAT_ID: str = "PUT_YOUR_CHAT_ID"

    # Backoff on throttle/empty frames
    BACKOFF_MIN_SEC: int = 60
    BACKOFF_MAX_SEC: int = 300

CFG = Config()

logging.basicConfig(
    level=CFG.LOG_LEVEL,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
log = logging.getLogger("live_ma_batch_alerts")

# =========================
# TIME HELPERS
# =========================
def now_ist() -> datetime:
    return datetime.now(tz=IST)

def is_market_open_ist(dt: Optional[datetime] = None) -> bool:
    d = dt or now_ist()
    if d.weekday() >= 5:  # Sat/Sun
        return False
    start = d.replace(hour=9, minute=15, second=0, microsecond=0)
    end   = d.replace(hour=15, minute=30, second=0, microsecond=0)
    return start <= d <= end

def today_market_end_ist() -> datetime:
    d = now_ist()
    return d.replace(hour=15, minute=30, second=0, microsecond=0)

# =========================
# TELEGRAM
# =========================
def send_telegram(msg: str) -> None:
    if not CFG.TELEGRAM_ENABLED:
        log.info(f"[DRY RUN] {msg}")
        return
    if not CFG.TELEGRAM_BOT_TOKEN or not CFG.TELEGRAM_CHAT_ID or \
       "PUT_YOUR" in CFG.TELEGRAM_BOT_TOKEN or "PUT_YOUR" in CFG.TELEGRAM_CHAT_ID:
        log.warning("Telegram credentials not set; printing instead.")
        log.info(msg)
        return
    try:
        url = f"https://api.telegram.org/bot{CFG.TELEGRAM_BOT_TOKEN}/sendMessage"
        data = {"chat_id": CFG.TELEGRAM_CHAT_ID, "text": msg, "parse_mode": "Markdown"}
        r = requests.post(url, data=data, timeout=10)
        if r.status_code != 200:
            log.warning(f"Telegram send failed: {r.status_code} {r.text}")
    except Exception as e:
        log.exception(f"Telegram error: {e}")

# =========================
# STATE (debounce)
# =========================
def load_state() -> Dict:
    if os.path.exists(CFG.STATE_FILE):
        try:
            with open(CFG.STATE_FILE, "r") as f:
                return json.load(f)
        except Exception:
            return {}
    return {}

def save_state(state: Dict) -> None:
    try:
        with open(CFG.STATE_FILE, "w") as f:
            json.dump(state, f, indent=2, sort_keys=True)
    except Exception as e:
        log.warning(f"Failed to save state: {e}")

# side: +1=above, -1=below, 0=near (within band)
def side_vs_ma(price: float, ma: float, eps_band: float) -> int:
    if ma is None or math.isnan(ma) or ma == 0:
        return 0
    up_th = ma * (1 + eps_band)
    dn_th = ma * (1 - eps_band)
    if price >= up_th:
        return +1
    elif price <= dn_th:
        return -1
    else:
        return 0

def fmt_pct(x: float) -> str:
    return f"{x:.2f}%"

# =========================
# YAHOO HELPERS (BATCHED)
# =========================
def batch_download_daily(symbols: List[str], lookback_days: int) -> Dict[str, pd.DataFrame]:
    """
    Returns dict: symbol -> daily OHLCV DataFrame
    Uses one batched call; handles both single/multi ticker shapes.
    """
    joined = " ".join(symbols)
    df = yf.download(
        tickers=joined,
        period=f"{lookback_days}d",
        interval="1d",
        auto_adjust=False,
        group_by="ticker",
        progress=False,
        threads=True,
    )
    out: Dict[str, pd.DataFrame] = {}
    # If only one ticker, columns won't be multi-level
    if isinstance(df.columns, pd.MultiIndex):
        for sym in symbols:
            if sym in df.columns.get_level_values(0):
                sub = df[sym].dropna(how="all")
                if not sub.empty:
                    out[sym] = sub
    else:
        out[symbols[0]] = df.dropna(how="all")
    return out

def batch_download_1m_last_close(symbols: List[str]) -> Dict[str, Optional[float]]:
    """
    Returns dict: symbol -> last 1m close (float) or None.
    One batched request for all symbols.
    """
    joined = " ".join(symbols)
    df = yf.download(
        tickers=joined,
        period="1d",
        interval="1m",
        group_by="ticker",
        auto_adjust=False,
        progress=False,
        threads=True,
    )
    prices: Dict[str, Optional[float]] = {s: None for s in symbols}

    if df is None or df.empty:
        return prices

    if isinstance(df.columns, pd.MultiIndex):
        top_syms = df.columns.get_level_values(0).unique().tolist()
        for sym in symbols:
            if sym in top_syms:
                sub = df[sym]
                if sub is not None and not sub.empty and "Close" in sub:
                    try:
                        prices[sym] = float(sub["Close"].dropna().iloc[-1])
                    except Exception:
                        prices[sym] = None
    else:
        # single symbol case
        try:
            prices[symbols[0]] = float(df["Close"].dropna().iloc[-1])
        except Exception:
            prices[symbols[0]] = None
    return prices

def compute_mas_from_daily(daily_map: Dict[str, pd.DataFrame], mas: List[int]) -> Dict[str, Dict[int, float]]:
    ma_cache: Dict[str, Dict[int, float]] = {}
    for sym, df in daily_map.items():
        if df is None or df.empty or "Close" not in df:
            continue
        closes = df["Close"].dropna()
        vals: Dict[int, float] = {}
        for n in mas:
            if len(closes) >= n:
                vals[n] = float(closes.tail(n).mean())
            else:
                vals[n] = float("nan")
        ma_cache[sym] = vals
    return ma_cache

# =========================
# MAIN
# =========================
def run():
    # Validate symbols
    symbols = [s.strip() for s in CFG.SYMBOLS if s.strip()]
    if not symbols:
        log.error("Please populate CFG.SYMBOLS with up to 50 NSE tickers (e.g., 'RELIANCE.NS').")
        return
    if len(symbols) > 50:
        log.warning("Yahoo batch works best up to ~50 tickers; extra will be ignored.")
        symbols = symbols[:50]

    state = load_state()
    end_today = today_market_end_ist()
    log.info(f"Watching {len(symbols)} symbols until {end_today.strftime('%H:%M IST')}")

    # Initial daily MA load (batched)
    backoff = CFG.BACKOFF_MIN_SEC
    while True:
        try:
            daily_map = batch_download_daily(symbols, CFG.DAILY_LOOKBACK_DAYS)
            if not daily_map:
                raise RuntimeError("Empty daily batch")
            ma_cache = compute_mas_from_daily(daily_map, CFG.MAS)
            # Initialize state using yesterday close vs MA to avoid instant false alerts
            for sym in symbols:
                state.setdefault(sym, {})
                df = daily_map.get(sym)
                if df is None or df.empty: 
                    continue
                prev_close = float(df["Close"].dropna().iloc[-1])
                for n in CFG.MAS:
                    s0 = side_vs_ma(prev_close, ma_cache.get(sym, {}).get(n, float("nan")), CFG.EPS_BAND)
                    state[sym].setdefault(str(n), {"side": s0, "last_alert": None})
            save_state(state)
            break
        except Exception as e:
            log.warning(f"Daily init failed: {e}. Backing off {backoff}s.")
            time.sleep(backoff)
            backoff = min(CFG.BACKOFF_MAX_SEC, int(backoff * 1.5 + random.randint(0, 10)))

    last_daily_refresh = now_ist()
    poll = CFG.POLL_SECONDS

    while True:
        now = now_ist()
        if now > end_today or not is_market_open_ist(now):
            log.info("Market closed or outside hours. Exiting.")
            break

        # Refresh daily MA cache every DAILY_REFRESH_MIN minutes
        if (now - last_daily_refresh) >= timedelta(minutes=CFG.DAILY_REFRESH_MIN):
            try:
                daily_map = batch_download_daily(symbols, CFG.DAILY_LOOKBACK_DAYS)
                if daily_map:
                    ma_cache = compute_mas_from_daily(daily_map, CFG.MAS)
                    last_daily_refresh = now
                    log.info("Refreshed daily MAs.")
            except Exception as e:
                log.warning(f"Daily refresh failed: {e}")

        # Batch fetch last 1m close for all symbols
        try:
            prices = batch_download_1m_last_close(symbols)
            if not any(v is not None for v in prices.values()):
                # Likely throttle or holiday; back off a bit and retry
                sleep_s = random.randint(CFG.BACKOFF_MIN_SEC, CFG.BACKOFF_MIN_SEC + 60)
                log.warning(f"No intraday prices returned; backing off {sleep_s}s.")
                time.sleep(sleep_s)
                continue
        except Exception as e:
            sleep_s = random.randint(CFG.BACKOFF_MIN_SEC, CFG.BACKOFF_MIN_SEC + 60)
            log.warning(f"Intraday batch failed: {e}. Backing off {sleep_s}s.")
            time.sleep(sleep_s)
            continue

        # Evaluate crossovers
        for sym in symbols:
            price = prices.get(sym)
            mas = ma_cache.get(sym, {})
            if price is None or not mas:
                continue
            sym_state = state.setdefault(sym, {})
            for n in CFG.MAS:
                ma = mas.get(n, float("nan"))
                new_side = side_vs_ma(price, ma, CFG.EPS_BAND)
                ent = sym_state.setdefault(str(n), {"side": 0, "last_alert": None})
                old_side = ent.get("side", 0)

                crossed_up = (old_side in (-1, 0)) and (new_side == +1)
                crossed_dn = (old_side in (+1, 0)) and (new_side == -1)

                if crossed_up or crossed_dn:
                    direction = "▲ Crossed *UP* through" if crossed_up else "▼ Crossed *DOWN* through"
                    gap_pct = (price - ma) / ma * 100.0 if (ma and not math.isnan(ma)) else float("nan")
                    ts = now.strftime("%Y-%m-%d %H:%M:%S %Z")
                    msg = (
                        f"*{sym}* {direction} *{n}D MA*\n"
                        f"Price: `{price:.2f}` | MA{n}: `{ma:.2f}` | Δ: {fmt_pct(gap_pct)}\n"
                        f"Time: {ts}"
                    )
                    send_telegram(msg)
                    ent["last_alert"] = now.isoformat()

                ent["side"] = new_side

        save_state(state)

        # Poll interval with small jitter to avoid patterns
        jitter = random.randint(-5, 5)
        time.sleep(max(10, poll + jitter))

    log.info("Stopped. State saved to %s", CFG.STATE_FILE)

# =========================
# ENTRY
# =========================
if __name__ == "__main__":
    run()


2025-10-16 14:40:23 | WARNING | Yahoo batch works best up to ~50 tickers; extra will be ignored.


2025-10-16 14:40:23 | INFO | Watching 50 symbols until 15:30 IST
2025-10-16 14:46:45 | INFO | [DRY RUN] *SHREECEM.NS* ▲ Crossed *UP* through *150D MA*
Price: `30160.00` | MA150: `30115.41` | Δ: 0.15%
Time: 2025-10-16 14:46:44 IST
2025-10-16 14:50:42 | INFO | [DRY RUN] *DRREDDY.NS* ▼ Crossed *DOWN* through *200D MA*
Price: `1238.60` | MA200: `1240.25` | Δ: -0.13%
Time: 2025-10-16 14:50:41 IST
2025-10-16 14:52:45 | INFO | [DRY RUN] *MAXHEALTH.NS* ▲ Crossed *UP* through *150D MA*
Price: `1173.20` | MA150: `1171.17` | Δ: 0.17%
Time: 2025-10-16 14:52:44 IST
2025-10-16 15:02:53 | INFO | [DRY RUN] *SHREECEM.NS* ▼ Crossed *DOWN* through *150D MA*
Price: `30005.00` | MA150: `30115.41` | Δ: -0.37%
Time: 2025-10-16 15:02:51 IST
2025-10-16 15:10:46 | INFO | Refreshed daily MAs.
2025-10-16 15:10:47 | INFO | [DRY RUN] *DRREDDY.NS* ▲ Crossed *UP* through *150D MA*
Price: `1241.50` | MA150: `1239.58` | Δ: 0.16%
Time: 2025-10-16 15:10:44 IST
2025-10-16 15:10:47 | INFO | [DRY RUN] *DRREDDY.NS* ▲ Crossed